## Importing the necessary libraries
* working with the python version 3.10.10

In [1]:
import gradio as gr
import numpy as np
from keras.models import load_model

### Importing Custom Modules

In [2]:
from utility.draw import Draw
from utility.process import Process

### Loading Model

In [3]:
model = load_model("trained model\\model 0")

#### Setup the variables for our model

In [4]:
actions = np.array([label for label in open('labels.txt', 'r').read().splitlines()])
process = Process()
draw = Draw()
sequence, sentence = [], ""
threshold = 0.99

In [5]:
def clear(text):
    global sentence
    sentence = ""
    return sentence

def pred(frame,show_keypoint=True):
    global sequence, sentence
    frame, results  = process.mediapipeDetection(frame)
    if results.pose_landmarks:
        if show_keypoint:
            draw.drawStyledLandmarks(frame, results)
        keypoints = process.extractKeypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(
                sequence, axis=0), verbose=0)[0]
            if res[np.argmax(res)] > threshold:
                sentence += " "+actions[np.argmax(res)]
                sequence.clear()
            else:
                # sentence = ""
                pass
    return sentence , frame


isvisible = False
def keypoints(image):
    global isvisible
    isvisible = not isvisible
    return gr.update(visible=isvisible)

In [6]:
with gr.Blocks() as app:
    with gr.Row():
        gr.Label("تطبيق ترجمة لغة الاشارة للصم والبكم")
    with gr.Row():
        with gr.Column():
            webcam = gr.Webcam(shape=(640,480),visible=True)
            show_keypoint = gr.Button("اظهار نقاط التعرف")
            image = gr.Webcam(shape=(640,480),visible=False)
        with gr.Column():
            translated = gr.Textbox(label="الترجمة",lines=15)
            clearbtn = gr.Button("مسح")

    clearbtn.click(clear,inputs=translated,outputs=translated)
    show_keypoint.click(keypoints,inputs=image,outputs=image)
    webcam.stream(pred,inputs=webcam,outputs=[translated,image])

app.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://a5df1c131f164ff014.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
